In [1]:
%matplotlib inline

import os
import sys
import random
import tables as tb
import numpy  as np

import matplotlib as mpl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple

import invisible_cities.core.fit_functions  as fitf
from invisible_cities.icaro.hst_functions import hist

import invisible_cities.reco.paolina_functions as plf
import invisible_cities.reco.peak_functions as pf
import invisible_cities.reco.xy_algorithms as xy
from   invisible_cities.reco    .dst_functions   import load_xy_corrections
from   invisible_cities.reco    .corrections     import LifetimeCorrection
import invisible_cities.reco    .tbl_functions as tbl

from   invisible_cities.database import load_db
from   invisible_cities.core.configure         import configure
from   invisible_cities.core.exceptions import SipmEmptyList
from   invisible_cities.core.exceptions import ClusterEmptyList
from   invisible_cities.core.exceptions import SipmZeroCharge

from   invisible_cities.io.pmap_io             import load_pmaps
from   invisible_cities.io.table_io            import make_table
from   invisible_cities.reco                   import pmaps_functions  as pmp
from   invisible_cities.reco    .xy_algorithms import barycenter
from   invisible_cities.reco    .xy_algorithms import corona
from   invisible_cities.evm.event_model        import Hit, Cluster
from   invisible_cities.evm.nh5                import HitsTable
from   invisible_cities.types.ic_types         import xy
from   invisible_cities.reco.tbl_functions     import get_event_numbers_and_timestamps_from_file_name
from   invisible_cities.core.system_of_units_c import units

from   invisible_cities.filters.s1s2_filter    import s1s2_filter
from   invisible_cities.filters.s1s2_filter    import s2si_filter
from   invisible_cities.filters.s1s2_filter    import S12Selector

###  Configuration parameters

In [2]:
e_lifetime = 1200.
drift_v = 1. 
run_number    = -4529

tot_charge = 60
n_rebin = 2
corona_opts   = {
    "Qthr"          : 0.,
    "Qlm"           : 15.,
    "lm_radius"     : 20.,
    "new_lm_radius" : 25.,
    "msipm"         :  6,
}

In [3]:
DataSiPM        = load_db.DataSiPM(run_number)
data_xs         = DataSiPM.X.values
data_ys         = DataSiPM.Y.values

### Corona and other useful functions

In [4]:
def get_nearby_sipm_inds(cs, d, pos, qs):
    """return indices of sipms less than d from (xc,yc)"""
    return np.where(np.linalg.norm(pos - cs, axis=1) <= d)[0]

def get_closest(cs, d, pos, qs):
    """return indices of sipms in a corona around (xc,yc)"""
    closest = []
    idx = 0
    for p in pos:
        if 9.999 < np.absolute(p[0] - cs[0]) < 10.001 or np.absolute(p[0] - cs[0]) < 0.001:
            if 9.999 < np.absolute(p[1] - cs[1]) < 10.001 or np.absolute(p[1] - cs[1]) < 0.001:
                closest.append(idx)
        idx += 1
    return closest

def discard_sipms(sis, pos, qs):
    return np.delete(pos, sis, axis=0), np.delete(qs, sis)

def my_corona(pos, qs, Qthr        =  3 * units.pes,
                    Qlm            =  15 * units.pes,
                    lm_radius      = 21.0 * units.mm,
                    new_lm_radius  = 31.0 * units.mm,
                    msipm          =  4):
    """
    corona creates a list of Clusters by
    first , identifying a max sipm
    second, calling barycenter to find the Cluster given by SiPMs around the max
    third , removing (nondestructively) the sipms contributing to that Cluster
    until there are no more local maxima
    kwargs
    Qthr : SiPMs with less than Qthr pes are ignored
    Qlm  : local maxima must have a SiPM with at least T pes
    msipm: the minimum number of SiPMs needed to make a cluster
    returns
    c    : a list of Clusters
    """
    c  = []
    # Keep SiPMs with at least Qthr pes
    above_threshold = np.where(qs > Qthr)[0]
    pos, qs = pos[above_threshold], qs[above_threshold]    
    
    # While there are more local maxima
    ncenter = 0
    while len(qs) > 0:
        print('Tot charge in slice: {}'.format(qs.sum()))
        hottest_sipm = np.argmax(qs)       # SiPM with largest Q
      
        if qs[hottest_sipm] < Qlm: break   # largest Q remaining is negligible

        # find locmax (the baryc of charge in SiPMs less than lm_radius from hottest_sipm)
        ring = get_closest(pos[hottest_sipm], lm_radius, pos, qs)
        
        #print('Number of SiPMs inside around hottest: {}'.format(len(ring) -1) )
        #print (pos[ring])

        if len(ring) -1  >= msipm:
            c.extend(barycenter(pos[ring],
                                qs [ring]))
        
        # delete the SiPMs contributing to this cluster
        pos, qs = discard_sipms(within_new_lm_radius, pos, qs)

    return c

def alex_corona(pos, qs, Qthr        =  3 * units.pes,
                    Qlm            =  15 * units.pes,
                    lm_radius      = 21.0 * units.mm,
                    new_lm_radius  = 31.0 * units.mm,
                    msipm          =  4):
    """
    corona creates a list of Clusters by
    first , identifying a loc max (gonz wanted more precise than just max sipm)
    second, calling barycenter to find the Cluster given by SiPMs around the max
    third , removing (nondestructively) the sipms contributing to that Cluster
    until there are no more local maxima
    kwargs
    Qthr : SiPMs with less than Qthr pes are ignored
    Qlm  : local maxima must have a SiPM with at least T pes
    lm_radius  : all SiPMs within lm_radius distance from the local max
           SiPM are used (by barycenter) to compute the approximate center
            of the local max.
    new_lm_radius : xs,ys,qs, of SiPMs within new_lm_radius of a local max
           are used by barycenter to compute a Cluster.
    msipm: the minimum number of SiPMs needed to make a cluster
    returns
    c    : a list of Clusters
    """
    c  = []
    # Keep SiPMs with at least Qthr pes
    above_threshold = np.where(qs >= Qthr)[0]
    pos, qs = pos[above_threshold], qs[above_threshold]    
    
    # While there are more local maxima
    ncenter = 0
    while len(qs) > 0:
        hottest_sipm = np.argmax(qs)       # SiPM with largest Q

        if qs[hottest_sipm] < Qlm: break   # largest Q remaining is negligible

        # find locmax (the baryc of charge in SiPMs less than lm_radius from hottest_sipm)
        within_lm_radius = get_nearby_sipm_inds(pos[hottest_sipm], lm_radius, pos, qs)

        new_local_maximum  = barycenter(pos[within_lm_radius], qs [within_lm_radius])[0].posxy[0]
        
        ## new_lm_radius is an array of the responsive sipms less than
        ## new_lm_radius from locmax
        within_new_lm_radius = get_nearby_sipm_inds(new_local_maximum,
                                                    new_lm_radius, pos, qs)

        ## if there are at least msipms within_new_lm_radius, get the barycenter
        if len(within_new_lm_radius) >= msipm:
            c.extend(barycenter(pos[within_new_lm_radius],
                                qs [within_new_lm_radius]))


        # delete the SiPMs contributing to this cluster
        pos, qs = discard_sipms(within_new_lm_radius, pos, qs)

    return c



In [5]:
def writer(output_file):
                        
    hits_table  = make_table(output_file,
                             group       = 'RECO',
                             name        = 'Events',
                             fformat     = HitsTable,
                             description = 'Hits',
                             compression = 'ZLIB4')

    hits_table.cols.event.create_index()
    row = hits_table.row
    
    def write_hits(hitc, evt_id):
        for slc in hitc:
            for hit in slc:
                row["event"] = evt_id
  #              row["time" ] = self.time
                row["npeak"] = hit.npeak
                row["nsipm"] = hit.nsipm
                row["X"    ] = hit.X
                row["Y"    ] = hit.Y
                row["Xrms" ] = hit.Xrms
                row["Yrms" ] = hit.Yrms
                row["Z"    ] = hit.Z
                row["Q"    ] = hit.Q
                row["E"    ] = hit.E
                row.append()
    return write_hits

## Loop to build hits and write them to output file, to be read afterwards, if needed

In [6]:
PATH_IN  = '/Users/paola/Software/ic_data/pair_1592keV_z250mm_1000evt_pmaps_1.pes_0pes.root.h5'
corrections   = "/Users/paola/Software/ic_data/MCmap.h5"
XYcorrection  = load_xy_corrections(corrections)
output_file = '/Users/paola/Software/ic_data/pair_1592keV_z250mm_1000evt_hits_1.pes_0pes.root.h5'

In [ ]:
hitc_evt = []

peak_energy = []
corr_peak_energy = []
charge_per_slice = []
charge_max_sipm = []

hitc_mc = []

zero_energy = 0

with tb.open_file(output_file, "w",
                          filters = tbl.filters('ZLIB4')) as h5out: 
    write_hits = writer(h5out)

    S1s, S2s, S2Sis = load_pmaps(PATH_IN)
    event_numbers, timestamps = get_event_numbers_and_timestamps_from_file_name(PATH_IN)
    for evt_number in event_numbers:
        print('Event {}'.format(evt_number))
        hitc = []

        s1   = S1s  .get(evt_number, {})
        s2raw = S2s  .get(evt_number, {})
        siraw = S2Sis.get(evt_number, {})

        ### Reject events with no S1, S2 or SiPM response in S2 
        if(not s1 or not s2raw or not siraw): continue
        if(len(s2raw.s2d) != len(siraw.s2sid)): continue
        if (len(s1.s1d) != 1): continue

        ### Rebibn S2 if needed
        s2, s2si = pmp.rebin_s2si(s2raw, siraw, n_rebin)

        s1t, s1e = next(iter(s1.s1d.values()))
        S1t  = s1t[np.argmax(s1e)]
           
        for peak_no, (t, E) in sorted(s2.s2d.items()):
            peak_energy.append(np.sum(E))
            #Container for cathod energies for slices with hits
            cath_energy = []

            si = s2si.s2sid[peak_no]
            nslices = len(s2.s2d[peak_no][0])

            e_left = 0
            for slice_no in range(nslices):
                hits_slice = []
                sipm_content = {sipm_no: sipm[slice_no] for sipm_no, sipm in si.items()}
                IDs, Qs = map(list, zip(*sipm_content.items()))
                Qs = np.array(Qs)            
                charge_per_slice.append(np.sum(Qs))
                
                if np.sum(Qs) < tot_charge:                
                    e_left += E[slice_no]
                else:
                    pos = np.array([np.array([x,y]) for x,y in zip(data_xs[IDs],data_ys[IDs])])
                    try:
                        clusters = alex_corona(pos, Qs, **corona_opts)                  
                    except ClusterEmptyList:
                        clusters = barycenter(possipm, qsipm)

                    drift_time      = (t[slice_no] - S1t) / 1000.
                    z               = drift_time * drift_v 
                    e_corrLT        = (E[slice_no] + e_left) * np.exp(drift_time/e_lifetime) 

                    qsum = sum(cls.Q for cls in clusters)
                    for cluster in clusters:
                        c_energy = e_corrLT * cluster.Q / qsum * XYcorrection(cluster.X, cluster.Y).value[0]
                        hit      = Hit(peak_no, cluster, z, c_energy)
                        hits_slice.append(hit)
                    cath_energy.append(E[slice_no] + e_left)
                    e_left = 0
                    hitc.append(hits_slice)
            if(len(cath_energy) > 0 and e_left > 0):
                cath_energy_last = cath_energy[-1]
                cath_energy_last += e_left
                cath_energy[-1] = cath_energy_last

            ## redistribution of energy in the last slice with hits
                last_slice = hitc[-1]
                qsum_last = sum(h.Q for h in last_slice)
                for hit in last_slice:
                    e_corrLT_last = cath_energy_last * np.exp(hit.Z/drift_v/e_lifetime)
                    hit.energy = e_corrLT_last * hit.Q / qsum_last * XYcorrection(hit.X, hit.Y).value[0]
            corr_e_peak = 0.
            for slc in hitc:
                for hit in slc:
                    corr_e_peak += hit.energy
            if corr_e_peak == 0.:
                zero_energy += 1
            corr_peak_energy.append(corr_e_peak)

        hitc_evt.append(hitc)
        write_hits(hitc, evt_number)
        
print('S2s with no reconstructed hits = {}'.format(zero_energy))




### Print hits

In [ ]:
for hitc in hitc_evt:
    for slc in hitc:
        for hit in slc:
            print("{0} {1} {2} {3}".format(hit.X,hit.Y,hit.Z,hit.E)) 

### Control plots

In [ ]:
fig = plt.figure(1)
fig.set_figheight(5.0)
fig.set_figwidth(15.0)

ax1 = fig.add_subplot(121);
plt.hist(corr_peak_energy,bins=400,label='Corrected energy')
lnd = plt.legend(loc=1)
#plt.scatter(z_spec,e_spec,marker='.')
#plt.ylim([0,15])
plt.xlim([0,600000])
plt.xlabel('Energy (pes)')
plt.ylabel('Counts/bin')
plt.yscale('log', nonposy='clip')

In [ ]:
ax2 = fig.add_subplot(121);
plt.hist(peak_energy,bins=100,label='Raw energy')
lnd = plt.legend(loc=1)
#plt.scatter(z_spec,e_spec,marker='.')
#plt.xlim([50000,150000])
#plt.ylim([0,20])
plt.xlabel('Energy (pes)')
plt.ylabel('Counts/bin')
plt.yscale('log', nonposy='clip')

In [ ]:
fig = plt.figure(1)
fig.set_figheight(5.0)
fig.set_figwidth(15.0)

ax1 = fig.add_subplot(121);
plt.hist(l_qsipm_ccenter,bins=600,label='Charge of SiPM with max q')
lnd = plt.legend(loc=1)
#plt.scatter(z_spec,e_spec,marker='.')
#plt.ylim([0,135])
#plt.xlim([0,100])
plt.xlabel('Charge (pes)')
plt.ylabel('Counts/bin')
plt.yscale('log', nonposy='clip')

In [ ]:
fig = plt.figure(1)
fig.set_figheight(5.0)
fig.set_figwidth(15.0)

ax1 = fig.add_subplot(121);
plt.hist(charge_per_slice,bins=600,label='Charge per slice')
lnd = plt.legend(loc=1)
#plt.scatter(z_spec,e_spec,marker='.')
#plt.ylim([0,15])
plt.xlim([0,600])
plt.xlabel('Charge (pes)')
plt.ylabel('Counts/bin')

### This is to plot a single slice, to inspect it visually and see if the algorithm works as expected

In [ ]:
# plot a 48x48 SiPM map
# -- carried over from NEW_kr_diff_mc_train.ipynb
def plot_test_event(l_X,l_Y,l_Q,l_X0,l_Y0):
    """
    Plots a SiPM map in the NEW geometry
    """

    # set up the figure
    fig = plt.figure();
    ax1 = fig.add_subplot(111);
    fig.set_figheight(15.0)
    fig.set_figwidth(20.0)
    ax1.axis([-250, 250, -250, 250]);

    # plot the SiPM pattern
    plt.scatter(l_X,l_Y,c=l_Q)
    plt.colorbar()

    # place an X for reconstructed cluster positions
    ax1.scatter(l_X0,l_Y0,marker='x',s=100,color='red')
        
    plt.xlabel("x (mm)");
    plt.ylabel("y (mm)");

In [ ]:
def compute_xy_position(si, slice_no):
    si      = {sipm_no: sipm[slice_no] for sipm_no, sipm in si.items()} #pmp.select_si_slice(si, slice_no)
    IDs, Qs = map(list, zip(*si.items()))
    Qs = np.array(Qs)
    pos = np.array([np.array([x,y]) for x,y in zip(data_xs[IDs],data_ys[IDs])])
    return my_corona(pos, Qs, **corona_opts)

In [ ]:
### Parameters to change slice by slice
slice_no = 2
evt_no = 40118084
peak_no = 0
n_rebin = 2


S1s, S2s, S2Sis = load_pmaps(PATH_IN)

S2raw = S2s  .get(evt_no, {})
S1 = S1s  .get(evt_no, {})
Siraw = S2Sis.get(evt_no, {})

S2, S2si = pmp.rebin_s2si(S2raw, Siraw, n_rebin)

si = S2si.s2sid[peak_no]

t = S1.s1d[peak_no][0]
e = S1.s1d[peak_no][1]
S1t  = t[np.argmax(e)]

t_slice = S2.s2d[peak_no][0][slice_no]
z        = (t_slice - S1t) * units.ns * 0.001  # drift_v = 1.0

clusters = compute_xy_position(si, slice_no)
print("Found {0} clusters at t {1}".format(len(clusters), t_slice))

x0vals = []; y0vals = []
for c in clusters:
    x0vals.append(c.pos[0])
    y0vals.append(c.pos[1])
    print("cluster ({0},{1},{2})".format(c.pos[0],c.pos[1],z))
    
xvals = []; yvals = []; qvals = []

sQ = [s for s in si.items() if s[1][slice_no] > 0]

for s in sQ:
    xvals.append(data_xs[s[0]])
    yvals.append(data_ys[s[0]])
    qvals.append(s[1][slice_no])



plot_test_event(xvals,yvals,qvals,x0vals,y0vals)


In [ ]:
### Print the charge and position of each SiPM in the slice 
### and the sum
mysum = 0
for sipm in sQ: 
    print(sipm[0], data_xs[sipm[0]], data_ys[sipm[0]], sipm[1][slice_no])
    mysum += sipm[1][slice_no]
print('Total charge in slice {} = {}'.format(slice_no, mysum))

## Paolina analysis

### Paolina parameters

In [ ]:
vol_min = np.array([-250, -250, 0],dtype=np.int16)  # volume minimum (x,y,z)
vol_max = np.array([250, 250, 600],dtype=np.int16)  # volume maximum (x,y,z)
vox_size = np.array([10,10,10],dtype=np.int16)    # voxel size
blob_radius = 20.       

In [ ]:
# run Paolina for many events
trk_energies = []
number_of_tracks = []
number_of_voxels = []
e_main = []
l_eblob1 = []; l_eblob2 = []

emin = 300000
emax = 400000

for nevt in range(len(hitc_evt)):
    
    hitc = hitc_evt[nevt]
    print("Event {0}".format(nevt))
    
    hits = []
    for slc in hitc:
        for h in slc:
            hits.append(h)
    
    voxels = plf.voxelize_hits(hits,vox_size)
    trks = plf.make_track_graphs(voxels,vox_size)
    number_of_tracks.append(len(trks))
    for t in trks:
        etrk = sum([vox.E for vox in t.nodes()])
        trk_energies.append(etrk)
        number_of_voxels.append(len(t.nodes()))
    itmax = np.argmax([len(t) for t in trks])
    print("Found {0} tracks; max containing {1} voxels; total of {2} voxels".format(len(trks),len(trks[itmax]),len(voxels)))

    e_main = sum([vox.E for vox in trks[itmax].nodes()])
    energy_main.append(e_main)
    
    if emin < e_main < emax:
        continue
    
    eblobs = plf.blob_energies(trks[itmax],blob_radius)
    iter_eblobs = iter(eblobs)
    Eblob1, Eblob2 = next(iter_eblobs),next(iter_eblobs)

    # ensure blob1 always has higher energy
    if(Eblob2 > Eblob1):
        eswap = Eblob1
        Eblob1 = Eblob2
        Eblob2 = eswap

    # add distribution information (event-scale)
    l_eblob1.append(Eblob1)
    l_eblob2.append(Eblob2)
#    print(Eblob1, Eblob2)
    # get the extremes
    distances = plf.shortest_paths(trks[itmax])
    a,b = plf.find_extrema(distances)

l_eblob1 = np.array(l_eblob1)
l_eblob2 = np.array(l_eblob2)


In [ ]:
# Plot Eblob1 vs. Eblob2
# 2D histogram
fig = plt.figure(3);
fig.set_figheight(5.0);
fig.set_figwidth(7.5);

hxy, xxy, yxy = np.histogram2d(l_eblob2, l_eblob1, (30, 30), ((0, 150000), (0, 150000)))
extent1 = [0, 180000, 0, 180000]
sp1 = plt.imshow(hxy, extent=extent1, interpolation='none', aspect='auto', origin='lower', cmap='jet')
plt.xlabel("Eblob1 (Q)")
plt.ylabel("Eblob2 (Q)")
#plt.xlim([0,150000])
#plt.ylim([0,150000])
plt.colorbar()

In [ ]:
fig = plt.figure(1)
fig.set_figheight(5.0)
fig.set_figwidth(15.0)

ax1 = fig.add_subplot(121);
plt.hist(number_of_tracks,bins=10,range=(-0.5,9.5),label='Number of tracks')
lnd = plt.legend(loc=1)
#plt.scatter(z_spec,e_spec,marker='.')
#plt.ylim([100000,900000])
#plt.xlim([20000,160000])
plt.xlabel('Number of tracks per event')
plt.ylabel('Events/bin')

In [ ]:
fig = plt.figure(1)
fig.set_figheight(5.0)
fig.set_figwidth(15.0)

ax1 = fig.add_subplot(121);
plt.hist(number_of_voxels,bins=60,range=(-0.5,59.5),label='Number of voxels per track')
lnd = plt.legend(loc=1)
#plt.scatter(z_spec,e_spec,marker='.')
#plt.ylim([100000,900000])
#plt.xlim([20000,160000])
plt.xlabel('Number of voxels per track')
plt.ylabel('Events/bin')

In [ ]:
fig = plt.figure(1)
fig.set_figheight(10.0)
fig.set_figwidth(35.0)

ax1 = fig.add_subplot(121);
y, x, _ = hist(trk_energies,bins=10000,label='Energy of tracks',new_figure=False)
lnd = plt.legend(loc=2)
#plt.scatter(z_spec,e_spec,marker='.')
#plt.ylim([5000,7000])
plt.xlim([5000,7000])
plt.xlabel('Energy (pes)')
plt.ylabel('Events/bin')

seed = 10000, 340000, 1000
#f    = fitf.fit(fitf.gauss, x, y, seed, fit_range=(340000, 347000))
#print(f.values)
xmin = min(x)
xmax = max(x)
#x = np.linspace(xmin, xmax, 10000)
#plt.plot(x, f.fn(x), 'r-')

#plt.savefig('TrackEnergy_default.png')


In [ ]:
xrays = [x for x in trk_energies if x < 10000]
fig = plt.figure(1)
#fig.set_figheight(10.0)
#fig.set_figwidth(35.0)

plt.hist(xrays,bins=100,label='Energy of tracks')
plt.xlabel('Energy (pes)')
plt.ylabel('Events/bin')